# Código Mestrado

https://stackabuse.com/python-for-nlp-working-with-facebook-fasttext-library/

https://github.com/PacktPublishing/fastText-Quick-Start-Guide/blob/master/chapter5/python%20fastText%20unsupervised%20learning.ipynb

## Bibliotecas

In [1]:
import pandas as pd
import re
import os
import joblib
import collections
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy
import nltk
from rouge_score import rouge_scorer
from collections import Counter
import fasttext
import numpy as np
import itertools
import string
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
import math
from fasttext import load_model
from nltk.tokenize import word_tokenize

In [2]:
Word2Vec()

## Leitura de Base

### DataBase

In [3]:
papers = joblib.load('papers')

In [4]:
len(papers)

707

papers_treino = dict(itertools.islice(papers.items(),495))

In [5]:
papers_treino = dict(list(papers.items())[0:495])
papers_teste = dict(list(papers.items())[495:708])

### Padrão Ouro

In [6]:
padrao_ouro = joblib.load('selecao_padrao_ouro')

In [7]:
padrao_ouro_treino = dict(list(padrao_ouro.items())[0:495])
padrao_ouro_teste  = dict(list(padrao_ouro.items())[495:708])

In [143]:
len(padrao_ouro_treino)

495

# Pré-Processamento

In [8]:
def preprocessamento(corpus: str):
    """ Pré-processamento inicial para remover pontuações, padronizar a caixa e pegar o conteúdo de cada paper"""
    regIter = re.finditer('(<PAPER>(.*)<\/PAPER>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    #lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        #txt = re.sub(r'<.*?>','', txt)
        txt = re.sub("\\t","",txt)
        #txt = re.sub('\/','', txt)
        txt = re.sub("\-","",txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        #txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('&amp;quot;', '', txt)
        txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;\?@\[\]^_`{|}\~\%\.]', '', txt)
        txt = re.sub('=', ' ', txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        #txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
        txt = re.sub('i\.e.', '', txt)
        txt = re.sub('acc\.', '', txt)
        txt = re.sub('e\.g\.', '', txt)
    return txt

In [9]:
def pre_processamento_ouro(txt):
    #txt = re.sub('\\n',' ', txt)
    txt = txt.lower()
    #txt = re.sub(r'<.*?>','', txt)
    txt = re.sub("\\t","",txt)
    #txt = re.sub('\/','', txt)
    txt = re.sub("\-","",txt)
    txt = txt.strip()
    txt = re.sub(' +', ' ', txt)
    txt = re.sub("\(.*?\)","",txt)
    txt = re.sub('&amp;quot;', '', txt)
    txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
    txt = re.sub('\.+', '', txt)
    txt = re.sub('[!"\#\$\%\&\'()\*\+,:;=\?@\[\]^_`{|}\~\%\.]', '', txt)
    txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
    txt = re.sub(r'\d', '', txt) #removendo demais numeros
    #txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
    txt = re.sub('i\.e.', '', txt)
    txt = re.sub('acc\.', '', txt)
    txt = re.sub('e\.g\.', '', txt)
    return txt

In [10]:
papers_treino2 = {k: preprocessamento(v) for k, v in papers_treino.items()}
padrao_ouro_treino2 = {k: pre_processamento_ouro(v) for k, v in padrao_ouro_treino.items()}

In [11]:
papers_teste2 = {k: preprocessamento(v) for k, v in papers_teste.items()}
padrao_ouro_teste2 = {k: pre_processamento_ouro(v) for k, v in padrao_ouro_teste.items()}

In [12]:
papers_treino2[13]

'<s sid >a practical partofspeech tagger</s> <abstract> <s sid  ssid >we present an implementation of a partofspeech tagger based on a hidden markov model</s> <s sid  ssid >the methodology enables robust and accurate tagging with few resource requirements</s> <s sid  ssid >only a lexicon and some unlabeled training text are required</s> <s sid  ssid >accuracy exceeds </s> <s sid  ssid >we describe implementation strategies and optimizations which result in highspeed operation</s> <s sid  ssid >three applications for tagging are described phrase recognition word sense disambiguation and grammatical function assignment</s> <s sid  ssid > desiderata many words are ambiguous in their part of speech</s> <s sid  ssid >for example tag can be a noun or a verb</s> <s sid  ssid >however when a word appears in the context of other words the ambiguity is often reduced in a tag is a partofspeech label the tag can only be a noun</s> <s sid  ssid >a tagger is a system that uses context to assign part

# Word Embedding

## Pré-Processamento para Embedding

In [13]:
def separa_frases(txt: str):
    """Separa as frases dos papers"""
    txt = txt.split('</s>')
    return txt

In [14]:
def separa_frases_ouro(txt: str):
    """Separa as frases dos papers"""
    txt = txt.split('\n')
    return txt

In [15]:
def preprocessamento_secundario(txt: str):
    """Remoção das demais tags após quebrar no </s>"""
    txt = re.sub(r'<.*?>','', txt)
    txt = re.sub('\/','', txt)
    txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
    txt = re.sub("\(.*?\)","",txt)
    txt = re.sub(' +', ' ', txt)
    return txt.strip()

In [16]:
def preprocessamento_secundario_ouro(txt: str):
    """Remoção das demais tags após quebrar no \n"""
    txt = re.sub(r'<.*?>','', txt)
    txt = re.sub('\\n',' ', txt)
    txt = re.sub('\/','', txt)
    txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
    txt = re.sub("\(.*?\)","",txt)
    txt = re.sub(' +', ' ', txt)
    return txt.strip()

In [17]:
def segundo_preprocessamento(dicionario):
    """input: dicionário
        output: frases pré-processadas"""
    frases_papers_treino={}
    for i,v in dicionario.items():
        x=[]
        for j in v:
            j=preprocessamento_secundario(j)
            if j!='': #remove itens nulos da lista
                x.append(j)
        frases_papers_treino[i]= x
    return frases_papers_treino

frases_papers_treino={}
for i,v in papers_treino3.items():
    x=[]
    for j in v:
        j=preprocessamento_secundario(j)
        if j!='': #remove itens nulos da lista
            x.append(j)
    frases_papers_treino[i]= x

In [18]:
papers_treino3 = {k: separa_frases(v) for k, v in papers_treino2.items()}
frases_papers_treino = segundo_preprocessamento(papers_treino3)

In [19]:
papers_teste3 = {k: separa_frases(v) for k, v in papers_teste2.items()}
frases_papers_teste = segundo_preprocessamento(papers_teste3)

### Input Word2Vec

In [20]:
lista_tokens=[]
for i,v in frases_papers_treino.items():
    for j in v:
        lista_tokens.append(j.split(' '))

### Input FastText

In [21]:
with open('preprocessado_treino_fasttext.txt', 'w', encoding='utf-8') as f:
    for lista in frases_papers_treino.values():
        for line in lista:
            f.write(line)
            f.write('\n')

## Word2Vec

### CBOW

In [22]:
model_w2v_cbow = Word2Vec(lista_tokens, min_count=1, epochs=100, sg=0)

In [23]:
len(model_w2v_cbow.wv.key_to_index)

46323

### SKIP-GRAM

In [24]:
model_w2v_sg = Word2Vec(lista_tokens, min_count=1, epochs=100, sg=1)

In [25]:
len(model_w2v_sg.wv.key_to_index)

46323

## Fast Text

### CBOW

In [26]:
model_ft_cbow = fasttext.train_unsupervised('preprocessado_treino_fasttext.txt', model='cbow', minCount=1, epoch=100)

In [27]:
model_ft_cbow.save_model('model_ft_cbow.bin')
load_model('model_ft_cbow.bin')

In [28]:
len(model_ft_cbow.get_words( on_unicode_error='ignore'))

46324

### SKIP-GRAM

In [29]:
model_ft_sg = fasttext.train_unsupervised('preprocessado_treino_fasttext.txt', model='skipgram', minCount=1, epoch=100)

In [30]:
model_ft_sg.save_model('model_ft_sg.bin')
load_model('model_ft_sg.bin')

In [31]:
len(model_ft_sg.words)

46324

len(model_ft_sg.get_words( on_unicode_error='ignore'))

print(model_ft_sg.get_subwords('tnt'))

In [32]:
#comparando os tokens dos 2 embeddings
lista_w2v=[]
for i in model_w2v_sg.wv.key_to_index:
    lista_w2v.append(i)
    
lista_ft=[]
for i in model_ft_sg.get_words( on_unicode_error='ignore'):
    lista_ft.append(i)
    
list(set(lista_ft)-set(lista_w2v))

['</s>']

# Medida de Similaridade

## Detecção das Seções 
Abstract, Introdução e Conclusão

In [33]:
def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<[Aa][Bb][sS][tT][Rr][aA][cC][tT]>)(.+?)(</[aA][bB][sS][tT][rR][aA][cC][tT]>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    return textos

In [34]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<section.*[Ii][nN][tT][rR][oO][dD][Uu][Cc][Tt][Ii][Oo][Nn].*?>(.*)<\/section>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    return textos

In [35]:
def preprocessa_conclusion(corpus: str):
    regIter1 = re.finditer('(<section.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/section>)', corpus, re.DOTALL|re.MULTILINE)
    textos1 = [ t.group(2) for t in regIter1]
    regIter2 = re.finditer('(<section.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/section>)', corpus, re.DOTALL|re.MULTILINE)
    textos2 = [ t.group(2) for t in regIter2]
    regIter3 = re.finditer('(<section.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Ee][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/section>)', corpus, re.DOTALL|re.MULTILINE)
    textos3 = [ t.group(2) for t in regIter3]
    textos = textos1
    for i in textos2:
        if i not in textos:
            textos.append(i)
    for i in textos3:
        if i not in textos:
            textos.append(i)
    return textos

In [36]:
abstract        = {k: preprocessa_abstract(v) for k, v in papers_treino2.items()}
abstract_frases = {k: separa_frases(j) for k, v in abstract.items() for j in v}
abstract_frases = segundo_preprocessamento(abstract_frases)
abstract_palavras = {k:[word_tokenize(f) for f in v] for k,v in abstract_frases.items()}

In [37]:
introducao = {k: preprocessa_intro(v) for k, v in papers_treino2.items()}
introducao_frases = {k: separa_frases(j) for k, v in introducao.items() for j in v}
introducao_frases = segundo_preprocessamento(introducao_frases)
introducao_palavras = {k:[word_tokenize(f) for f in v] for k,v in introducao_frases.items()}

In [38]:
conclusao = {k: preprocessa_conclusion(v) for k, v in papers_treino2.items()}
conclusao_frases = {k: separa_frases(j) for k, v in conclusao.items() for j in v}
conclusao_frases = segundo_preprocessamento(conclusao_frases)
conclusao_palavras = {k:[word_tokenize(f) for f in v] for k,v in conclusao_frases.items()}

Validei que tudo mede 495

## Aplicação do Word Embedding

`model_w2v_cbow`

`model_w2v_sg`

`model_ft_cbow`

`model_ft_sg`

In [39]:
def vetor_frases_w2v(sessao_palavras, modelo_embedding):
    """input: dicionário de sessão das palavras
        output: dicionário de vetores"""
    dicio={}
    for k,v in sessao_palavras.items():
        x=[]
        for lista_frase in v:
            soma=0
            for palavra in lista_frase:
                try:
                    soma = soma + modelo_embedding.wv[palavra]
                except:
                    soma = soma
            x.append(soma)
        dicio[k] = x
    return dicio

In [40]:
def vetor_frases_ft(sessao_palavras, modelo_embedding):
    """input: dicionário de sessão das palavras
        output: dicionário de vetores"""
    dicio={}
    for k,v in sessao_palavras.items():
        x=[]
        for lista_frase in v:
            soma=0
            for palavra in lista_frase:
                try:
                    soma = soma + modelo_embedding.get_word_vector(palavra)
                except:
                    soma=soma
            x.append(soma)
        dicio[k] = x
    return dicio

In [41]:
v_w2v_cbow_abstract   = vetor_frases_w2v(abstract_palavras,model_w2v_cbow)
v_w2v_cbow_introducao = vetor_frases_w2v(introducao_palavras,model_w2v_cbow)
v_w2v_cbow_conclusao  = vetor_frases_w2v(conclusao_palavras,model_w2v_cbow)

In [42]:
v_w2v_sg_abstract   = vetor_frases_w2v(abstract_palavras,model_w2v_sg)
v_w2v_sg_introducao = vetor_frases_w2v(introducao_palavras,model_w2v_sg)
v_w2v_sg_conclusao  = vetor_frases_w2v(conclusao_palavras,model_w2v_sg)

In [43]:
v_ft_cbow_abstract   = vetor_frases_ft(abstract_palavras,model_ft_cbow)
v_ft_cbow_introducao = vetor_frases_ft(introducao_palavras,model_ft_cbow)
v_ft_cbow_conclusao  = vetor_frases_ft(conclusao_palavras,model_ft_cbow)

In [44]:
v_ft_sg_abstract   = vetor_frases_ft(abstract_palavras,model_ft_sg)
v_ft_sg_introducao = vetor_frases_ft(introducao_palavras,model_ft_sg)
v_ft_sg_conclusao  = vetor_frases_ft(conclusao_palavras,model_ft_sg)

In [45]:
def funcao_sim_coseno(a,b):
    if ((numpy.linalg.norm(a)==0) |(numpy.linalg.norm(b)==0)):
        return 0
    else:
        valor_coseno = (numpy.dot(a, b))/(numpy.linalg.norm(a)* numpy.linalg.norm(b))
        return valor_coseno
    
def dicionario_similaridade(vetor_a,vetor_b):
    """Input: vetor correspondente às frases das seções a e b
    Output: 
    """
    dicionario={}
    for i in range(len(vetor_a)):
        for j in range(len(vetor_b)):
            dicionario[i,j]= funcao_sim_coseno(vetor_a[i],vetor_b[j])
    return dicionario

def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

In [46]:
def aplicando_dicionario_similaridade(v1,v2):
    """cálculo da similaridade, pegando os 3 vetores mais similares"""
    x = []
    for i,j in zip(range(len(v1)),range(len(v2))):
        x.append(dicionario_similaridade(v1[i], v2[j]))
        
    top_3_indices = []
    for i in range(len(x)):
        top_3_indices.append(sorted(x[i], key=x[i].get, reverse=True)[:3])
        
    lista_1=[]
    for i in top_3_indices:
        top3=[]
        for j in i:
            top3.append(j[0])
        lista_1.append(list(set(top3)))
        
    return x,top_3_indices,lista_1

In [47]:
def frases_interseccao(lista_1,lista_2):
    """comparando se os índices similares pertencem às duas listas (abstract e conclusão) e (abstract e intro) 
    e retorna os índices em comum"""
    indices_objetivo=[]
    for i,j in zip(lista_1,lista_2):
        indices_objetivo.append(list(set.intersection(*map(set,[i,j]))))
    return indices_objetivo

In [48]:
def get_frases_objetivo(indices_objetivo, idx_paper):
    """retorna a frase objetivo [do abstract] a partir dos indices"""
    frase_objetivo={}
    for idx,idx_f,idx_ind in zip(idx_paper,range(len(list(abstract_frases.values()))),indices_objetivo):
        lista_one=[]
        for j in idx_ind:
            lista_one.append(list(abstract_frases.values())[idx_f][j])
            #print(idx_f)
            #print(j)
            #print(frases_abstract[idx_f][j])
        frase_objetivo[idx] = lista_one
    return frase_objetivo

### Similaridade w2v cbow

In [49]:
#Abtract e Introdução
sim_ab_intro_cbow,indices_ab_intro_cbow,lista_idx_ab_intro_cbow =aplicando_dicionario_similaridade(list(v_w2v_cbow_abstract.values()),list(v_w2v_cbow_introducao.values()))

#Abtract e Conclusão
sim_ab_conc_cbow,indices_ab_conc_cbow,lista_idx_ab_c_cbow = aplicando_dicionario_similaridade(list(v_w2v_cbow_abstract.values()),list(v_w2v_cbow_conclusao.values()))

#Pega os Indices em comum
indices_objetivo_cbow = frases_interseccao(lista_idx_ab_intro_cbow,lista_idx_ab_c_cbow)

#Retorna as Frases correspondentes
frase_objetivo_cbow_w2v = get_frases_objetivo(indices_objetivo_cbow, papers_treino.keys())

In [50]:
joblib.dump(frase_objetivo_cbow_w2v,'frase_objetivo_w2v_cbow')

['frase_objetivo_w2v_cbow']

### Similaridade w2v skip-gram

In [51]:
#Abtract e Introdução
sim_ab_intro_sg,indices_ab_intro_sg,lista_idx_ab_intro_sg =aplicando_dicionario_similaridade(list(v_w2v_sg_abstract.values()),list(v_w2v_sg_introducao.values()))

#Abtract e Conclusão
sim_ab_conc_sg,indices_ab_conc_sg,lista_idx_ab_c_sg = aplicando_dicionario_similaridade(list(v_w2v_sg_abstract.values()),list(v_w2v_sg_conclusao.values()))

#Pega os Indices em comum
indices_objetivo_sg = frases_interseccao(lista_idx_ab_intro_sg,lista_idx_ab_c_sg)

#Retorna as Frases correspondentes
frase_objetivo_sg_w2v = get_frases_objetivo(indices_objetivo_sg, papers_treino.keys())

In [52]:
joblib.dump(frase_objetivo_sg_w2v,'frase_objetivo_w2v_sg')

['frase_objetivo_w2v_sg']

vai até 4 a 14, pois o abstract tem 5 frases e a conclusao tem 15

### Similaridade fasttext cbow

In [53]:
#Abtract e Introdução
sim_ab_intro_cbow_ft,indices_ab_intro_cbow_ft,lista_idx_ab_intro_cbow_ft =aplicando_dicionario_similaridade(list(v_ft_cbow_abstract.values()),list(v_ft_cbow_introducao.values()))

#Abtract e Conclusão
sim_ab_conc_cbow_ft,indices_ab_conc_cbow_ft,lista_idx_ab_c_cbow_ft = aplicando_dicionario_similaridade(list(v_ft_cbow_abstract.values()),list(v_ft_cbow_conclusao.values()))

#Pega os Indices em comum
indices_objetivo_cbow_ft = frases_interseccao(lista_idx_ab_intro_cbow_ft,lista_idx_ab_c_cbow_ft)

#Retorna as Frases correspondentes
frase_objetivo_cbow_ft  = get_frases_objetivo(indices_objetivo_cbow_ft, papers_treino.keys())

In [54]:
joblib.dump(frase_objetivo_cbow_ft,'frase_objetivo_cbow_ft')

['frase_objetivo_cbow_ft']

In [154]:
frase_objetivo_cbow_ft

{0: ['contrary to claims found elsewhere in the literature we argue that a tagger based on markov models performs at least as well as other current approaches including the maximum entropy framework',
  'a recent comparison has even shown that tnt performs significantly better for the tested corpora'],
 2: ['intersentence similarity is replaced by rank in the local context'],
 3: ['this paper presents a corpusbased approach to word sense disambiguation that builds an ensemble of naive bayesian classifiers each of which is based on lexical features that represent cooccurring words in varying sized windows of context'],
 4: ['we present a new parser for parsing down to penn treebank style parse trees that achieves average precisionrecall for sentences of and less and for of length and less when trained and tested on the previously established standard sections of the wall street journal treebank',
  'this represents a decrease in error rate over the best singleparser results on this corp

### Similaridade fasttext skip-gram 

In [55]:
#Abtract e Introdução
sim_ab_intro_sg_ft,indices_ab_intro_sg_ft,lista_idx_ab_intro_sg_ft =aplicando_dicionario_similaridade(list(v_ft_sg_abstract.values()),list(v_ft_sg_introducao.values()))

#Abtract e Conclusão
sim_ab_conc_sg_ft,indices_ab_conc_sg_ft,lista_idx_ab_c_sg_ft = aplicando_dicionario_similaridade(list(v_ft_sg_abstract.values()),list(v_ft_sg_conclusao.values()))

#Pega os Indices em comum
indices_objetivo_sg_ft = frases_interseccao(lista_idx_ab_intro_sg_ft,lista_idx_ab_c_sg_ft)

#Retorna as Frases correspondentes
frase_objetivo_sg_ft = get_frases_objetivo(indices_objetivo_sg_ft, papers_treino.keys())

In [56]:
frase_objetivo_sg_ft.keys()

dict_keys([0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 16, 18, 19, 20, 31, 32, 33, 35, 37, 39, 41, 43, 45, 47, 48, 50, 51, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 74, 88, 97, 99, 101, 102, 103, 104, 105, 106, 108, 110, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 136, 137, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 191, 192, 193, 194, 198, 200, 202, 203, 204, 205, 206, 207, 208, 209, 210, 222, 223, 225, 226, 228, 229, 231, 232, 234, 235, 243, 244, 245, 246, 248, 258, 263, 272, 276, 277, 279, 280, 281, 282, 283, 284, 286, 296, 307, 310, 313, 315, 316, 318, 323, 326, 338, 339, 342, 345, 347, 348, 350, 351, 352, 355, 356, 357, 358, 359, 362, 363, 364, 365, 366, 369, 370, 371, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382

In [57]:
joblib.dump(frase_objetivo_sg_ft,'frase_objetivo_sg_ft')

['frase_objetivo_sg_ft']

As frases que extrai foram da variavel `abstract_frases`, ou seja, são as frases do abstract já pré-processadas

## Validação - Exp1 - ROUGE-N

### Pré-Processamento Ouro

In [58]:
def separa_frases_ouro(txt: str):
    """Separa as frases dos papers"""
    txt = txt.split('\n')
    return txt

padrao_ouro_treino_frases = {k: separa_frases_ouro(v) for k, v in padrao_ouro_treino2.items()}
padrao_ouro_treino_frases = segundo_preprocessamento(padrao_ouro_treino_frases)
padrao_ouro_treino_palavras = {k:[word_tokenize(f) for f in v] for k,v in padrao_ouro_treino_frases.items()}

### Rouge

In [59]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [60]:
def calculo_rouge(frase_objetivo,lista_frases_padrao_ouro):
    rouge_total=[]
    for i,j in zip(frase_objetivo,lista_frases_padrao_ouro):
        rouge_1={}
        for h in range(len(i)):
            for z in range(len(j)):
                rouge_1[(h,z)] = (scorer.score(i[h],j[z])['rouge1'].fmeasure)
        rouge_total.append(rouge_1)
    return rouge_total

In [61]:
def get_max(dic, coord, val):
    return max(filter(lambda item: item[0][coord] == val, dic.items()), key=lambda x: x[1])

In [62]:
def max_rouge(rouge_total):
    """pega o máximo rouge score para cada frase selecionada como pertencente ao objetivo, em comparação com cada frase
    do padrão ouro"""
    maximo_rouge_por_paper =[]
    for h in rouge_total:
        x=[]
        for i in set([i[0] for i in h]):
            x.append(get_max(h,0,i))
        maximo_rouge_por_paper.append(x)
    return maximo_rouge_por_paper

In [63]:
rouge_sg_ft = calculo_rouge(list(frase_objetivo_sg_ft.values()),list(padrao_ouro_treino_frases.values()))

lista=[]
for idx in list(padrao_ouro_treino_frases.keys()):
    dicionario = {}
    for f_obj, idx_obj in zip(frase_objetivo_sg_ft[idx],range(len(frase_objetivo_sg_ft[idx]))):
        for f_ouro, idx_ouro in zip(padrao_ouro_treino_frases[idx], range(len(padrao_ouro_treino_frases[idx]))):
            dicionario[idx_obj, idx_ouro] = scorer.score(f_obj,f_ouro)['rouge1'].fmeasure
            #print(scorer.score(f_obj,f_ouro)['rouge1'].fmeasure)
        lista.append(dicionario)

dicionario = {}
for f_obj, idx_obj in zip(frase_objetivo_sg_ft[8],range(len(frase_objetivo_sg_ft[8]))):
    for f_ouro, idx_ouro in zip(padrao_ouro_treino_frases[8], range(len(padrao_ouro_treino_frases[8]))):
        dicionario[idx_obj, idx_ouro] = scorer.score(f_obj,f_ouro)['rouge1'].fmeasure
        print(scorer.score(f_obj,f_ouro)['rouge1'].fmeasure)

In [64]:
#w2v
rouge_sg_w2v = calculo_rouge(list(frase_objetivo_sg_w2v.values()),list(padrao_ouro_treino_frases.values()))
rouge_cbow_w2v = calculo_rouge(list(frase_objetivo_cbow_w2v.values()),list(padrao_ouro_treino_frases.values()))

#FT
rouge_sg_ft = calculo_rouge(list(frase_objetivo_sg_ft.values()),list(padrao_ouro_treino_frases.values()))
rouge_cbow_ft = calculo_rouge(list(frase_objetivo_cbow_ft.values()),list(padrao_ouro_treino_frases.values()))

In [65]:
#w2v
maximo_rouge_por_paper_sg_w2v   = max_rouge(rouge_sg_w2v)
maximo_rouge_por_paper_cbow_w2v = max_rouge(rouge_cbow_w2v)

#FT
maximo_rouge_por_paper_sg_ft   = max_rouge(rouge_sg_ft)
maximo_rouge_por_paper_cbow_ft   = max_rouge(rouge_cbow_ft)

In [66]:
def rouge_precisao_revocacao(maximo_rouge_por_paper, lista_padrao_ouro):
    precisao=[]
    for i in maximo_rouge_por_paper:
        soma=0
        for j in i:
            soma = soma + j[1]
        #print(len(i))
        try:
            precisao.append(soma/len(i))
        except:
            precisao.append(0)

    revocacao=[]
    for i,h in zip(maximo_rouge_por_paper,lista_padrao_ouro):
        soma=0
        for j in i:
            soma = soma + j[1]
        #print(len(i))
        try:
            revocacao.append(soma/len(h))
        except:
            revocacao.append(0)
    return precisao, revocacao

In [67]:
lista_padrao_ouro_treino  = list(padrao_ouro_treino_frases.values())

In [68]:
#w2v
precisao_sg_w2v, revocacao_sg_w2v = rouge_precisao_revocacao(maximo_rouge_por_paper_sg_w2v, lista_padrao_ouro_treino)
precisao_cbow_w2v, revocacao_cbow_w2v = rouge_precisao_revocacao(maximo_rouge_por_paper_cbow_w2v, lista_padrao_ouro_treino)

#FT
precisao_sg_ft, revocacao_sg_ft = rouge_precisao_revocacao(maximo_rouge_por_paper_sg_ft, lista_padrao_ouro_treino)
precisao_cbow_ft, revocacao_cbow_ft = rouge_precisao_revocacao(maximo_rouge_por_paper_cbow_ft, lista_padrao_ouro_treino)

In [69]:
precisao_sg_ft==precisao_cbow_ft

False

In [70]:
revocacao_cbow_ft==revocacao_sg_ft

False

In [71]:
np.mean(precisao_cbow_w2v)

0.8052397372035844

# Experimento 1 - Conclusão
Rouge Usado foi o N=1 e a precisão e recall foram calculadas a partir do f1 score

In [72]:
def analise_precisao(precisao, lista_frases_padrao_ouro):
    x=[]
    qtd_zero=[]
    for i in precisao:
        if i>=0.8:
            x.append(i)
        if i==0:
            qtd_zero.append(i)
    print('qtd de papers com precisao>=0.8:',len(x))
    print('% do total:',len(x)/len(lista_frases_padrao_ouro))
    print('qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções:',len(qtd_zero))
    print('% do total:',len(qtd_zero)/len(lista_frases_padrao_ouro))
    print('media precisao:', np.mean(precisao))
    return len(x), len(qtd_zero),np.mean(precisao)

In [73]:
def analise_revocacao(revocacao,lista_frases_padrao_ouro,media_ouro):
    x=[]
    qtd_zero=[]
    for i in revocacao:
        if i>=(1/media_ouro): #valor de 1/média qtd frases padrao ouro
            x.append(i)
        if i==0:
            qtd_zero.append(i)
    print('qtd de papers com revocação>=(1/media_ouro):',len(x))
    print('% do total:',len(x)/len(lista_frases_padrao_ouro))
    print('qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções:',len(qtd_zero))
    print('% do total:',len(qtd_zero)/len(lista_frases_padrao_ouro))
    return len(x), np.mean(revocacao)

In [74]:
qtd_precisao_08_sg_w2v, qtd_zero_sg_w2v, media_sg_w2v = analise_precisao(precisao_sg_w2v, lista_padrao_ouro_treino)

qtd de papers com precisao>=0.8: 403
% do total: 0.8141414141414142
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 59
% do total: 0.1191919191919192
media precisao: 0.8307540252735163


In [75]:
qtd_precisao_08_cbow_w2v, qtd_zero_cbow_w2v, media_cbow_w2v = analise_precisao(precisao_cbow_w2v, lista_padrao_ouro_treino)

qtd de papers com precisao>=0.8: 394
% do total: 0.795959595959596
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 74
% do total: 0.1494949494949495
media precisao: 0.8052397372035844


In [76]:
qtd_precisao_08_sg_ft, qtd_zero_sg_ft, media_sg_ft = analise_precisao(precisao_sg_ft, lista_padrao_ouro_treino)

qtd de papers com precisao>=0.8: 402
% do total: 0.8121212121212121
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 67
% do total: 0.13535353535353536
media precisao: 0.8207254113547606


In [142]:
qtd_precisao_08_cbow_ft, qtd_zero_cbow_ft, media_cbow_ft = analise_precisao(precisao_cbow_ft, lista_padrao_ouro_treino)

qtd de papers com precisao>=0.8: 409
% do total: 0.8262626262626263
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 59
% do total: 0.1191919191919192
media precisao: 0.831012788967217


In [78]:
precisao_mean =  {'media_cbow_ft':media_cbow_ft,
                  'media_sg_ft':media_sg_ft,
                  'media_cbow_w2v':media_cbow_w2v,
                  'media_sg_w2v':media_sg_w2v}
max(precisao_mean, key=precisao_mean.get)

'media_cbow_ft'

In [79]:
qtd_papers_hipotese_invalida = {'qtd_zero_sg_w2v':qtd_zero_sg_w2v,
                              'qtd_zero_cbow_w2v':qtd_zero_cbow_w2v,
                               'qtd_zero_cbow_ft':qtd_zero_cbow_ft,
                              'qtd_zero_sg_ft':qtd_zero_sg_ft}
min(qtd_papers_hipotese_invalida, key=qtd_papers_hipotese_invalida.get)

'qtd_zero_sg_w2v'

In [80]:
qtd_precisao_alta = {'qtd_precisao_08_sg_w2v':qtd_precisao_08_sg_w2v,
                     'qtd_precisao_08_sg_w2v':qtd_precisao_08_sg_w2v,
                     'qtd_precisao_08_cbow_ft':qtd_precisao_08_cbow_ft,
                     'qtd_precisao_08_sg_ft':qtd_precisao_08_sg_ft}
max(qtd_precisao_alta, key=qtd_precisao_alta.get)

'qtd_precisao_08_cbow_ft'

In [81]:
#Média de palavras do Padrão Ouro
x=[]
for i in lista_padrao_ouro_treino:
    x.append(len(i))
media_ouro=np.mean(x)
media_ouro

7.642424242424243

In [82]:
print('w2v')
print('skip-gram')
qtd_revocacao_sg_w2v, media_revocacao_sg_w2v     = analise_revocacao(revocacao_sg_w2v, lista_padrao_ouro_treino, media_ouro)

print('cbow')
qtd_revocacao_cbow_w2v,  media_revocacao_cbow_w2v   = analise_revocacao(revocacao_cbow_w2v, lista_padrao_ouro_treino, media_ouro)

print('-----------')
print('fasttext')
print('skip-gram')
qtd_revocacao_sg_ft,  media_revocacao_sg_ft       = analise_revocacao(revocacao_sg_ft, lista_padrao_ouro_treino, media_ouro)

print('cbow')
qtd_revocacao_cbow_ft,  media_revocacao_cbow_ft       = analise_revocacao(revocacao_cbow_ft, lista_padrao_ouro_treino, media_ouro)

w2v
skip-gram
qtd de papers com revocação>=(1/media_ouro): 312
% do total: 0.6303030303030303
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 59
% do total: 0.1191919191919192
cbow
qtd de papers com revocação>=(1/media_ouro): 296
% do total: 0.597979797979798
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 74
% do total: 0.1494949494949495
-----------
fasttext
skip-gram
qtd de papers com revocação>=(1/media_ouro): 312
% do total: 0.6303030303030303
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 67
% do total: 0.13535353535353536
cbow
qtd de papers com revocação>=(1/media_ouro): 325
% do total: 0.6565656565656566
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 59
% do total: 0.1191919191919192


In [83]:
qtd_revocacao_alta = {'qtd_revocacao_sg_w2v':qtd_revocacao_sg_w2v,
                      'qtd_revocacao_cbow_w2v':qtd_revocacao_cbow_w2v,
                     'qtd_revocacao_sg_ft':qtd_revocacao_sg_ft,
                     'qtd_revocacao_cbow_ft':qtd_revocacao_cbow_ft}
max(qtd_revocacao_alta, key=qtd_revocacao_alta.get)

'qtd_revocacao_cbow_ft'

In [84]:
media_revocacao   = {'media_revocacao_sg_w2v':media_revocacao_sg_w2v,
                     'media_revocacao_cbow_w2v':media_revocacao_cbow_w2v,
                     'media_revocacao_sg_ft':media_revocacao_sg_ft,
                     'media_revocacao_cbow_ft':media_revocacao_cbow_ft}
max(media_revocacao, key=media_revocacao.get)

'media_revocacao_sg_w2v'

In [138]:
media_revocacao_sg_w2v

0.19107567596404137

In [139]:
media_revocacao_cbow_w2v

0.1771586186176

In [140]:
media_revocacao_sg_ft

0.19093071913479925

In [141]:
media_revocacao_cbow_ft

0.1877424709115802

# Experimento 2

`frases_objetivo_modelo` alterar pro resultado oficial depois

In [85]:
frases_objetivo_modelo = frase_objetivo_cbow_ft.copy()
palavras_objetivo_modelo = {k:[word_tokenize(f) for f in v] for k,v in frases_objetivo_modelo.items()}

## Bigrama

In [86]:
#O bigrama é calculado a partir de uma lista com listas de palavras de cada frase
frase_objetivo_ft_formato_ml=[]
for i in list(palavras_objetivo_modelo.values()):
    for j in i:
        frase_objetivo_ft_formato_ml.append(j)

In [87]:
#qtd de frases objetivo total
len(frase_objetivo_ft_formato_ml)

697

In [88]:
#qtd de palavras de cada frase
len(frase_objetivo_ft_formato_ml[1])

15

In [89]:
#Bigrama - O treino do bigrama ocorre a partir das frases objetivo
n = 2
train_data, padded_sents = padded_everygram_pipeline(n, frase_objetivo_ft_formato_ml)

model = MLE(n)
model.fit(train_data, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 3190 items>


`frases_papers_treino`

In [90]:
palavras_papers_treino = {k:[word_tokenize(f) for f in v] for k,v in frases_papers_treino.items()}

In [91]:
#Criando os bigramas do paper de treino completo
bigramas=[]
for papers in list(palavras_papers_treino.values()):
    x=[]
    for frases in papers:
        x.append(bigrams(frases))
    bigramas.append(x)

In [92]:
lista_bigramas2=[]
for j in bigramas:
    lista_bigramas=[]  
    for i in j:
        lista_bigramas.append(list(i))
    lista_bigramas2.append(lista_bigramas)

In [93]:
# Probabilidade de cada frase do paper, com base nos bigramas
lista_paper_prob=[]
lista_paper_perp=[]
for paper in lista_bigramas2:
    lista_prob=[]
    lista_perp=[]
    for frase in paper:
        prob=1
        for bigrama in frase:
            prob = prob * (model.score(bigrama[1], context=bigrama[0].split()))
        lista_prob.append(prob)
        try:
            lista_perp.append(model.perplexity(frase))
        except:
            lista_perp.append(math.inf)
    lista_paper_prob.append(lista_prob)
    lista_paper_perp.append(lista_perp)

In [94]:
for i in lista_paper_perp:
    print(set(i))

{16.47238049749035, 12.648706266787169, inf}
{11.629476754092662, inf}
{11.847763389762102, inf}
{16.733996216525092, 15.749670881351575, inf}
{20.16839235199321, inf}
{inf}
{16.212859164282868, inf}
{16.58866139577263, 13.231838309201267, inf}
{17.368289659722418, inf}
{16.812348299400277, 15.371545281857356, inf}
{inf}
{19.18019637079806, 13.853269779872566, 22.71036541901987, inf}
{inf}
{16.086055411616357, inf}
{32.32604945658536, inf}
{26.829833593252555, 15.481870315743803, 278.9999999999999, inf}
{12.334862151433546, inf}
{44.05592806115084, 30.14331368427415, inf}
{1.5491933384829668, 11.926871292303703, 183.00000000000014, inf}
{18.29881480333431, 19.155512625666418, inf}
{4.999999999999999, 9.000000000000002, 12.648615984565062, 13.773275880747272, inf}
{8.259323732443463, 1.7500000000000002, 15.775330878389303, inf}
{1.0, 18.48392453281956, 14.771870172870226, inf}
{16.22758329523058, 20.60838439654403, inf}
{17.49478675041073, inf}
{144.69968901141425, 81.3142054010245, 10.

In [95]:
for i in lista_paper_prob:
    print(set(i))

{0.0, 8.681939995634711e-34, 9.234446772403475e-18}
{0.0, 2.570149734654029e-10}
{0.0, 2.64683560122929e-38}
{0.0, 1.450823815086472e-54, 1.73061012554813e-17}
{0.0, 8.064442902301553e-27}
{0.0}
{0.0, 7.324420256152657e-42}
{0.0, 1.4593197655664175e-27, 5.203014049537852e-31}
{0.0, 1, 3.6429874848987405e-08}
{0.0, 1.9609181041695747e-15, 3.302017036458673e-29}
{0.0, 1}
{0.0, 2.223526376731727e-54, 5.0805026833054306e-61, 2.822416263511938e-33}
{0.0}
{0.0, 2.8715056264536765e-27}
{0.0, 1.9924833743257123e-32}
{0.0, 2.2002500847419e-17, 1.929870127781526e-06, 0.0035842293906810036}
{0.0, 1, 1.2194940675195413e-23}
{0.0, 1, 7.859069556986966e-29, 1.1694642147111013e-05}
{0.0, 1, 0.4166666666666667, 1.221340290613607e-27, 0.00546448087431694}
{0.0, 1.1578587258581836e-19, 1.116572937516703e-18, 1}
{0.0, 1, 0.1111111111111111, 0.2, 1.526356358419e-09, 6.340789403705126e-27}
{0.0, 1, 0.5714285714285714, 4.617895358484145e-08, 4.4103545515167e-27}
{1.3170334487514217e-20, 0.0, 4.6347793845012

## Trigrama

In [96]:
#Trigrama
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, frase_objetivo_ft_formato_ml)

model_tri = MLE(n)
model_tri.fit(train_data, padded_sents)
print(model_tri.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 3190 items>


In [97]:
#Criando os bigramas da lista
trigramas=[]
for papers in list(palavras_papers_treino.values()):
    x=[]
    for frases in papers:
        x.append(trigrams(frases))
    trigramas.append(x)
    
lista_trigramas2=[]
for j in trigramas:
    lista_trigramas=[]  
    for i in j:
        lista_trigramas.append(list(i))
    lista_trigramas2.append(lista_trigramas)

In [98]:
# Probabilidade de cada frase do paper, com base nos bigramas
lista_paper_prob_tri=[]
lista_paper_perp_tri=[]
for paper in lista_trigramas2:
    lista_prob_tri=[]
    lista_perp_tri=[]
    for frase in paper:
        prob=1
        for trigrama in frase:
            prob = prob * (model_tri.score(trigrama[2], context=[trigrama[1],trigrama[0]]))
        lista_prob_tri.append(prob)
        try:
            lista_perp_tri.append(model_tri.perplexity(frase))
        except:
            lista_perp_tri.append(math.inf)
    lista_paper_prob_tri.append(lista_prob_tri)
    lista_paper_perp_tri.append(lista_perp_tri)

In [ ]:
lista_paper_perp_tri_filtrada=[]
for paper in lista_trigramas2:
    lista_prob_tri=[]
    lista_perp_tri=[]
    for frase in paper:
        prob=1
        for trigrama in frase:
            tri=model_tri.score(trigrama[2], context=[trigrama[0],trigrama[1]])
            if tri==0:
                lista_perp_tri.append(model_tri.perplexity(frase)) 
    lista_paper_perp_tri_filtrada.append(lista_perp_tri)

In [ ]:
lista_paper_perp_tri_filtrada

In [ ]:
for i in lista_paper_perp:
    print(set(i))

In [ ]:
for i in lista_paper_prob:
    print(set(i))

[1] Validar se não é infinita a perplexidade onde existe a frase objetivo

[2] Tirar a média da perplexidade de todos os papers

## Conclusão Experimento 2

In [101]:
lista_contador_verdadeiro=[]
lista_contador_falso=[]
for perplexidades_bi_paper, perplexidades_tri_paper in zip(lista_paper_perp,lista_paper_perp_tri):
    contador_positivo=0
    contador_negativo=0
    for p_bi, p_tri in zip(perplexidades_bi_paper, perplexidades_tri_paper):
        if p_tri<=p_bi:
            contador_positivo=contador_positivo+1
        else:
            contador_negativo=contador_negativo+1
        #lista_contador_negativo.append(contador_negativo)
    lista_contador_verdadeiro.append(contador_positivo)
    lista_contador_falso.append(contador_negativo)

In [144]:
len(lista_contador_verdadeiro)

495

In [102]:
lista_contador_verdadeiro

[179,
 148,
 157,
 190,
 192,
 178,
 129,
 110,
 128,
 223,
 288,
 107,
 143,
 149,
 81,
 169,
 173,
 219,
 266,
 158,
 205,
 201,
 141,
 187,
 181,
 207,
 120,
 201,
 219,
 155,
 228,
 188,
 302,
 197,
 221,
 224,
 205,
 251,
 323,
 212,
 225,
 132,
 195,
 139,
 244,
 134,
 163,
 251,
 271,
 260,
 284,
 260,
 265,
 183,
 409,
 117,
 117,
 212,
 307,
 227,
 138,
 187,
 161,
 293,
 193,
 167,
 130,
 167,
 166,
 203,
 217,
 128,
 167,
 239,
 199,
 154,
 189,
 146,
 250,
 252,
 197,
 187,
 182,
 196,
 184,
 226,
 143,
 150,
 182,
 229,
 154,
 207,
 199,
 216,
 203,
 194,
 226,
 230,
 259,
 151,
 176,
 155,
 188,
 220,
 202,
 166,
 146,
 151,
 138,
 132,
 139,
 154,
 167,
 167,
 187,
 148,
 153,
 209,
 144,
 229,
 158,
 206,
 186,
 173,
 194,
 228,
 73,
 154,
 204,
 107,
 173,
 222,
 275,
 204,
 196,
 193,
 179,
 226,
 192,
 205,
 266,
 187,
 155,
 92,
 195,
 438,
 501,
 570,
 145,
 522,
 415,
 564,
 404,
 350,
 494,
 399,
 545,
 605,
 538,
 382,
 218,
 736,
 276,
 380,
 460,
 387,
 286,
 

In [103]:
lista_contador_falso

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 1,
 0,
 2,
 1,
 2,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 1,
 0,
 2,
 1,
 0,
 1,
 1,
 1,
 3,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 2,
 0,
 2,
 2,
 4,
 2,
 2,
 3,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 3,
 1,
 2,
 2,
 0,
 1,
 1,
 0,
 0,
 5,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 3,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,


In [ ]:
lista_contador_verdadeiro=[]
lista_contador_falso=[]
for perplexidades_bi_paper, perplexidades_tri_paper in zip(lista_paper_perp,lista_paper_perp_tri):
    contador_positivo=0
    contador_negativo=0
    for p_bi, p_tri in zip(perplexidades_bi_paper, perplexidades_tri_paper):
        if p_tri<=p_bi:
            contador_positivo=contador_positivo+1
        else:
            contador_negativo=contador_negativo+1
        #lista_contador_negativo.append(contador_negativo)
    lista_contador_verdadeiro.append(contador_positivo)
    lista_contador_falso.append(contador_negativo)

TIRAR A MÉDIA DE TUDO E COMPARAR QUAL É MENOR

# Experimento 3

## Parte 1
Aplicando o método da similaridade nos 30%

### Similaridade

In [104]:
def vetor_frases_ft_teste(sessao_palavras, modelo_embedding):
    """input: dicionário de sessão das palavras
        output: dicionário de vetores"""
    dicio={}
    for k,v in sessao_palavras.items():
        x=[]
        for lista_frase in v:
            soma=0
            for palavra in lista_frase:
                try:
                    soma = soma + modelo_embedding.get_word_vector(palavra)
                except:
                    soma=soma
            x.append(soma)
        dicio[k] = x
    return dicio

In [105]:
modelo_selecionado = model_ft_cbow

In [106]:
abstract_teste        = {k: preprocessa_abstract(v) for k, v in papers_teste2.items()}
abstract_frases_teste = {k: separa_frases(j) for k, v in abstract_teste.items() for j in v}
abstract_frases_teste = segundo_preprocessamento(abstract_frases_teste)
abstract_palavras_teste = {k:[word_tokenize(f) for f in v] for k,v in abstract_frases_teste.items()}

introducao_teste = {k: preprocessa_intro(v) for k, v in papers_teste2.items()}
introducao_frases_teste = {k: separa_frases(j) for k, v in introducao_teste.items() for j in v}
introducao_frases_teste = segundo_preprocessamento(introducao_frases_teste)
introducao_palavras_teste = {k:[word_tokenize(f) for f in v] for k,v in introducao_frases_teste.items()}

conclusao_teste = {k: preprocessa_conclusion(v) for k, v in papers_teste2.items()}
conclusao_frases_teste = {k: separa_frases(j) for k, v in conclusao_teste.items() for j in v}
conclusao_frases_teste = segundo_preprocessamento(conclusao_frases_teste)
conclusao_palavras_teste = {k:[word_tokenize(f) for f in v] for k,v in conclusao_frases_teste.items()}

In [107]:
v_abstract_teste    = vetor_frases_ft_teste(abstract_palavras_teste,modelo_selecionado)
v_introducao_teste  = vetor_frases_ft_teste(introducao_palavras_teste,modelo_selecionado)
v_conclusao_teste   = vetor_frases_ft_teste(conclusao_palavras_teste,modelo_selecionado)

In [108]:
def get_frases_objetivo_teste(indices_objetivo, idx_paper):
    """retorna a frase objetivo [do abstract] a partir dos indices"""
    frase_objetivo={}
    for idx,idx_f,idx_ind in zip(idx_paper,range(len(list(abstract_frases_teste.values()))),indices_objetivo):
        lista_one=[]
        for j in idx_ind:
            lista_one.append(list(abstract_frases_teste.values())[idx_f][j])
        frase_objetivo[idx] = lista_one
    return frase_objetivo

In [109]:
#Abtract e Introdução
sim_ab_intro_teste,indices_ab_intro_teste,lista_idx_ab_intro_teste =aplicando_dicionario_similaridade(list(v_abstract_teste.values()),list(v_introducao_teste.values()))

#Abtract e Conclusão
sim_ab_conc_teste,indices_ab_conc_teste,lista_idx_ab_c_teste = aplicando_dicionario_similaridade(list(v_abstract_teste.values()),list(v_conclusao_teste.values()))

#Pega os Indices em comum
indices_objetivo_teste = frases_interseccao(lista_idx_ab_intro_teste,lista_idx_ab_c_teste)

In [110]:
#Retorna as Frases correspondentes
frase_objetivo_teste  = get_frases_objetivo_teste(indices_objetivo_teste, papers_teste.keys())

In [111]:
frase_objetivo_teste

{746: ['the algorithm is based on two powerful constraints that words tend to have one sense per discourse and one sense per collocation exploited in an iterative bootstrapping procedure'],
 747: ['to attack these problems we have built a hybrid generator in which gaps in symbolic knowledge are filled by statistical methods',
  'we also discuss how the hybrid generation model can be used to simplify current generators and enhance their portability even when perfect knowledge is in principle obtainable'],
 748: ['syntactic natural language parsers have shown themselves to be inadequate for processing highlyambiguous largevocabulary text as is evidenced by their poor performance on domains like the wall street journal and by the movement away from parsingbased approaches to textprocessing in general'],
 749: ['this study suggests that the identification of word translations should also be possible with nonparallel and even unrelated texts',
  'the method proposed is based on the assumpti

### ROUGE-N

In [112]:
padrao_ouro_teste_frases = {k: separa_frases_ouro(v) for k, v in padrao_ouro_teste2.items()}
padrao_ouro_teste_frases = segundo_preprocessamento(padrao_ouro_teste_frases)
padrao_ouro_teste_palavras = {k:[word_tokenize(f) for f in v] for k,v in padrao_ouro_teste_frases.items()}

In [113]:
lista_padrao_ouro_teste = list(padrao_ouro_teste_frases.values())

In [114]:
#w2v
rouge_teste = calculo_rouge(list(frase_objetivo_teste.values()),(padrao_ouro_teste_frases.values()))

In [115]:
#Média de palavras do Padrão Ouro
x=[]
for i in lista_padrao_ouro_treino:
    x.append(len(i))
media_ouro_teste=np.mean(x)
media_ouro_teste

7.642424242424243

In [116]:
maximo_rouge_por_paper_ft_teste   = max_rouge(rouge_teste)


precisao_sg_ft_teste, revocacao_sg_ft_teste = rouge_precisao_revocacao(maximo_rouge_por_paper_ft_teste, lista_padrao_ouro_teste)
qtd_precisao_ft_teste, qtd_zero_ft_teste, media_ft_teste = analise_precisao(precisao_sg_ft_teste, lista_padrao_ouro_teste)
print('------')
qtd_revocacao_ft,  media_revocacao_ft       = analise_revocacao(revocacao_sg_ft_teste, lista_padrao_ouro_teste, media_ouro_teste)

qtd de papers com precisao>=0.8: 187
% do total: 0.8820754716981132
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 11
% do total: 0.05188679245283019
media precisao: 0.8928227222746559
------
qtd de papers com revocação>=(1/media_ouro): 159
% do total: 0.75
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 11
% do total: 0.05188679245283019


## Parte 2

### Modelo de Linguagem
Aplicando o modelo de linguagem nos 30%

Escolhido: trigrama

In [117]:
palavras_papers_teste = {k:[word_tokenize(f) for f in v] for k,v in frases_papers_teste.items()}

In [118]:
#Criando os bigramas da lista
trigramas=[]
for papers in list(palavras_papers_teste.values()):
    x=[]
    for frases in papers:
        x.append(trigrams(frases))
    trigramas.append(x)
    
lista_trigramas2=[]
for j in trigramas:
    lista_trigramas=[]  
    for i in j:
        lista_trigramas.append(list(i))
    lista_trigramas2.append(lista_trigramas)

In [119]:
# Probabilidade de cada frase do paper, com base nos trigramas
lista_paper_prob_tri=[]
lista_paper_perp_tri=[]
for paper in lista_trigramas2:
    lista_prob_tri=[]
    lista_perp_tri=[]
    for frase in paper:
        prob=1
        for trigrama in frase:
            prob = prob * (model_tri.score(trigrama[2], context=[trigrama[0],trigrama[1]])) #to na dúvida se é 1 e 0 ou 0 e 1
        lista_prob_tri.append(prob)
        try:
            lista_perp_tri.append(model_tri.perplexity(frase))
        except:
            lista_perp_tri.append(math.inf)
    lista_paper_prob_tri.append(lista_prob_tri)
    lista_paper_perp_tri.append(lista_perp_tri)
    
for i in lista_paper_prob:
    print(set(i))

{0.0, 8.681939995634711e-34, 9.234446772403475e-18}
{0.0, 2.570149734654029e-10}
{0.0, 2.64683560122929e-38}
{0.0, 1.450823815086472e-54, 1.73061012554813e-17}
{0.0, 8.064442902301553e-27}
{0.0}
{0.0, 7.324420256152657e-42}
{0.0, 1.4593197655664175e-27, 5.203014049537852e-31}
{0.0, 1, 3.6429874848987405e-08}
{0.0, 1.9609181041695747e-15, 3.302017036458673e-29}
{0.0, 1}
{0.0, 2.223526376731727e-54, 5.0805026833054306e-61, 2.822416263511938e-33}
{0.0}
{0.0, 2.8715056264536765e-27}
{0.0, 1.9924833743257123e-32}
{0.0, 2.2002500847419e-17, 1.929870127781526e-06, 0.0035842293906810036}
{0.0, 1, 1.2194940675195413e-23}
{0.0, 1, 7.859069556986966e-29, 1.1694642147111013e-05}
{0.0, 1, 0.4166666666666667, 1.221340290613607e-27, 0.00546448087431694}
{0.0, 1.1578587258581836e-19, 1.116572937516703e-18, 1}
{0.0, 1, 0.1111111111111111, 0.2, 1.526356358419e-09, 6.340789403705126e-27}
{0.0, 1, 0.5714285714285714, 4.617895358484145e-08, 4.4103545515167e-27}
{1.3170334487514217e-20, 0.0, 4.6347793845012

In [120]:
model_tri.score('presents', context=['paper','this'])

0

In [121]:
model_tri.score('presents', context=['this','paper'])

0.16161616161616163

In [122]:
prob_frase_teste=[]
for frase,prob in zip(list(frases_papers_teste.values()),lista_paper_prob_tri):
    dicionario={}
    for vl_frase, vl_prob in zip(frase,prob):
        #if vl_prob!=1:
            dicionario[vl_frase]= vl_prob
    prob_frase_teste.append(dicionario)

In [123]:
len(prob_frase_teste)

212

In [124]:
selecao_frase_objetivo_teste =[]
for h in prob_frase_teste:
    x=[]
    for i in set([i[0] for i in h]):
        x.append(get_max(h,0,i))
    selecao_frase_objetivo_teste.append(x)

In [125]:
frase_objetivo_teste_exp2=[]
for i in selecao_frase_objetivo_teste:
    try:
        frase_objetivo_teste_exp2.append([i[0][0]])
    except:
        frase_objetivo_teste_exp2.append([])

In [126]:
def calculo_rouge(frases,ouro):
    rouge_total=[]
    for i,j in zip(frases,ouro):
        rouge_1={}
        for h in range(len(i)):
            for z in range(len(j)):
                rouge_1[(h,z)] = (scorer.score(i[h],j[z])['rouge1'].fmeasure)
        rouge_total.append(rouge_1)
    return rouge_total

In [127]:
rouge_total=[]
for i,j in zip(frase_objetivo_teste_exp2,list(padrao_ouro_teste_frases.values())):
    #print (len(i))
    rouge_1={}
    for h in range(len(i)):
        for z in range(len(j)):
            rouge_1[(h,z)] = (scorer.score(i[h],j[z])['rouge1'].fmeasure)
    rouge_total.append(rouge_1)

In [128]:
#Média de palavras do Padrão Ouro
x=[]
for i in lista_padrao_ouro_treino:
    x.append(len(i))
media_ouro_teste=np.mean(x)
media_ouro_teste

7.642424242424243

In [129]:
frase_objetivo_teste_exp2

[['for example'],
 ['furthermore lexical constraints are not limited to the syntagmatic interlexical constraints discussed above'],
 ['where m lt n and k lt ki lt n for all i lt m for example a model pf ihi hh might be interpolated as follows where e ai hi hh for all histories hi hh'],
 ['word a in the english matrix is then the translation of word a in the german matrix'],
 ['for example in machine translation knowing the correct word sense helps to select the appropriate target words to use in order to translate into a target language'],
 ['for each leaf ie lexeme of a given syntax tree the lexicon specifies a lexical interpretation from the model'],
 ['finally we give experimental results in section using these two algorithms in appropriate domains and compare them to the labelled tree viterbi algorithm showing that each algorithm generally works best when evaluated on the criterion that it optimizes'],
 ['first the statistical model assigns a probability to every candidate parse tr

In [130]:
padrao_ouro_teste_frases[748]

['statistical decisiontree models for parsing',
 'syntactic natural language parsers have shown themselves to be inadequate for processing highlyambiguous largevocabulary text as is evidenced by their poor performance on domains like the wall street journal and by the movement away from parsingbased approaches to textprocessing in general',
 'in this paper i describe spatter a statistical parser based on decisiontree learning techniques which constructs a complete parse for every sentence and achieves accuracy rates far better than any published result',
 'this work is based on the following premises grammars are too complex and detailed to develop manually for most interesting domains parsing models must rely heavily on lexical and contextual information to analyze sentences accurately and existing ngram modeling techniques are inadequate for parsing models',
 'in experiments comparing spatter with ibms computer manuals parser spatter significantly outperforms the grammarbased parser'

In [131]:
rouge_total

[{(0, 0): 0.0,
  (0, 1): 0.06666666666666667,
  (0, 2): 0.0,
  (0, 3): 0.0,
  (0, 4): 0.0,
  (0, 5): 0.0},
 {(0, 0): 0.0,
  (0, 1): 0.13793103448275862,
  (0, 2): 0.13793103448275862,
  (0, 3): 0.0,
  (0, 4): 0.11764705882352941,
  (0, 5): 0.0,
  (0, 6): 0.15,
  (0, 7): 0.08695652173913043},
 {(0, 0): 0.09523809523809523,
  (0, 1): 0.10256410256410257,
  (0, 2): 0.11594202898550725,
  (0, 3): 0.12195121951219512,
  (0, 4): 0.0,
  (0, 5): 0.075,
  (0, 6): 0.03703703703703704,
  (0, 7): 0.10714285714285714},
 {(0, 0): 0.2608695652173913,
  (0, 1): 0.24242424242424243,
  (0, 2): 0.2222222222222222,
  (0, 3): 0.39024390243902435,
  (0, 4): 0.22857142857142856,
  (0, 5): 0.34146341463414637},
 {(0, 0): 0.15789473684210525,
  (0, 1): 0.27272727272727276,
  (0, 2): 0.19672131147540983,
  (0, 3): 0.10714285714285715,
  (0, 4): 0.19672131147540983,
  (0, 5): 0.14814814814814814,
  (0, 6): 0.05,
  (0, 7): 0.09523809523809523},
 {(0, 0): 0.07692307692307693,
  (0, 1): 0.05405405405405406,
  (0, 2

In [132]:
maximo_rouge_paper = max_rouge(rouge_total)

In [133]:
precisao_exp3_pt2, revocacao_exp3_pt2 = rouge_precisao_revocacao(maximo_rouge_paper, lista_padrao_ouro_teste)

In [134]:
np.mean(precisao_exp3_pt2)

0.5408221127312582

In [135]:
np.mean(revocacao_exp3_pt2)

0.08636530958097381

In [136]:
print('------ revocação ---------')
qtd_revocacao_exp3_pt2, media_revocacao_exp3_pt2  = analise_revocacao(revocacao_exp3_pt2, lista_padrao_ouro_teste, media_ouro_teste)

------ revocação ---------
qtd de papers com revocação>=(1/media_ouro): 57
% do total: 0.2688679245283019
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 8
% do total: 0.03773584905660377


In [137]:
print('------ precisao ---------')
precisao_cbow_exp3_pt2, revocacao_cbow_exp3_pt2 = analise_precisao(precisao_exp3_pt2, lista_padrao_ouro_teste)

------ precisao ---------
qtd de papers com precisao>=0.8: 84
% do total: 0.39622641509433965
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 8
% do total: 0.03773584905660377
media precisao: 0.5408221127312582


ValueError: too many values to unpack (expected 2)

In [ ]:
len(lista_padrao_ouro_teste)